https://docs.python.org/zh-cn/3/library/collections.html#chainmap-objects    

一个 ChainMap 类是为了将多个映射快速的链接到一起，这样它们就可以作为一个单元处理。**它通常比创建一个新字典和多次调用 update() 要快很多**。

这个类可以用于模拟嵌套作用域，并且在模版化的时候比较有用。

In [1]:
from collections import ChainMap

In [2]:
baseline = {'music': 'bach', 'art': 'rembrandt'}
adjustments = {'art': 'van gogh', 'opera': 'carmen'}
chainMap = ChainMap(adjustments, baseline)
chainMap

ChainMap({'art': 'van gogh', 'opera': 'carmen'}, {'music': 'bach', 'art': 'rembrandt'})

In [3]:
chainMap.keys()

KeysView(ChainMap({'art': 'van gogh', 'opera': 'carmen'}, {'music': 'bach', 'art': 'rembrandt'}))

In [4]:
chainMap.values()

ValuesView(ChainMap({'art': 'van gogh', 'opera': 'carmen'}, {'music': 'bach', 'art': 'rembrandt'}))

In [5]:
chainMap['art']

'van gogh'

In [6]:
list(chainMap.keys())

['music', 'art', 'opera']

In [7]:
list(chainMap.values())

['bach', 'van gogh', 'carmen']

In [8]:
# 这给出了与 dict.update() 调用序列相同的顺序，从最后一个映射开始:
combined = baseline.copy()
combined.update(adjustments)
list(combined)

['music', 'art', 'opera']

In [10]:
# 模拟Python内部lookup链的例子
import builtins
pylookup = ChainMap(locals(), globals(), vars(builtins))
pylookup

ChainMap({'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'from collections import ChainMap', "baseline = {'music': 'bach', 'art': 'rembrandt'}\nadjustments = {'art': 'van gogh', 'opera': 'carmen'}\nchainMap = ChainMap(adjustments, baseline)\nchainMap", 'chainMap.keys()', 'chainMap.values()', "chainMap['art']", 'list(chainMap.keys())', 'list(chainMap.values())', '# 这给出了与 dict.update() 调用序列相同的顺序，从最后一个映射开始:\ncombined = baseline.copy()\ncombined.update(adjustments)\nlist(combined)', '# 模拟Python内部lookup链的例子\nimport builtins\npylookup = ChainMap(locals(), globals(), vars(builtins))', '# 模拟Python内部lookup链的例子\nimport builtins\npylookup = ChainMap(locals(), globals(), vars(builtins))\npylookup'], '_oh': {2: ChainMap({'art': 'van gogh', 'opera': 'carmen'}, {'music': 'bach', 'ar

In [14]:
! python chain_map_test.py -c blue

blue
guest


In [18]:
# 用 ChainMap 类模拟嵌套上下文的例子
c = ChainMap({'a': '1', 'b': '2'})        # Create root context
d = c.new_child()     # Create nested child context
e = c.new_child()     # Child of c, independent from d
print(c)
print(d)
print(e)


ChainMap({'a': '1', 'b': '2'})
ChainMap({}, {'a': '1', 'b': '2'})
ChainMap({}, {'a': '1', 'b': '2'})


In [19]:
print(e.maps[0])             # Current context dictionary -- like Python's locals()
print(e.maps[-1])            # Root context -- like Python's globals()
print(e.parents)             # Enclosing context chain -- like Python's nonlocals



{}
{'a': '1', 'b': '2'}
ChainMap({'a': '1', 'b': '2'})


In [20]:
d['x'] = 1            # Set value in current context
print(d['x'])                # Get first key in the chain of contexts
del d['x']            # Delete from current context
print(list(d))               # All nested values
print('x' in d)                # Check all nested values
print(len(d))                # Number of nested values
print(d.items())             # All nested items
print(dict(d))               # Flatten into a regular dictionary

1
['a', 'b']
False
2
ItemsView(ChainMap({}, {'a': '1', 'b': '2'}))
{'a': '1', 'b': '2'}


In [21]:
# ChainMap 类只更新链中的第一个映射，但lookup会搜索整个链。 然而，如果需要深度写和删除，也可以很容易的通过定义一个子类来实现它
class DeepChainMap(ChainMap):
    'Variant of ChainMap that allows direct updates to inner scopes'

    def __setitem__(self, key, value):
        for mapping in self.maps:
            if key in mapping:
                mapping[key] = value
                return
        self.maps[0][key] = value

    def __delitem__(self, key):
        for mapping in self.maps:
            if key in mapping:
                del mapping[key]
                return
        raise KeyError(key)

d = DeepChainMap({'zebra': 'black'}, {'elephant': 'blue'}, {'lion': 'yellow'})
print(d)
d['lion'] = 'orange'         # update an existing key two levels down
print(d)
d['snake'] = 'red'           # new keys get added to the topmost dict
print(d)
del d['elephant']            # remove an existing key one level down
print(d)                          # display result

DeepChainMap({'zebra': 'black'}, {'elephant': 'blue'}, {'lion': 'yellow'})
DeepChainMap({'zebra': 'black'}, {'elephant': 'blue'}, {'lion': 'orange'})
DeepChainMap({'zebra': 'black', 'snake': 'red'}, {'elephant': 'blue'}, {'lion': 'orange'})
DeepChainMap({'zebra': 'black', 'snake': 'red'}, {}, {'lion': 'orange'})


In [22]:
d = ChainMap({'zebra': 'black'}, {'elephant': 'blue'}, {'lion': 'yellow'})
print(d)
d['lion'] = 'orange'         # update an existing key two levels down
print(d)
d['snake'] = 'red'           # new keys get added to the topmost dict
print(d)
del d['elephant']            # remove an existing key one level down
print(d)         

ChainMap({'zebra': 'black'}, {'elephant': 'blue'}, {'lion': 'yellow'})
ChainMap({'zebra': 'black', 'lion': 'orange'}, {'elephant': 'blue'}, {'lion': 'yellow'})
ChainMap({'zebra': 'black', 'lion': 'orange', 'snake': 'red'}, {'elephant': 'blue'}, {'lion': 'yellow'})


KeyError: "Key not found in the first mapping: 'elephant'"